In [32]:
# Import Splinter, BeautifulSoup, and Pandas
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
import datetime as dt

In [55]:
def scrape_all():
    # Set the executable path and initialize Splinter
    executable_path = {'executable_path': ChromeDriverManager().install()}
    browser = Browser('chrome', **executable_path, headless=False)
    news_title, news_paragraph = mars_news(browser)
    facts = mars_facts()
    data = {
      "news_title": news_title,
      "news_paragraph": news_paragraph,
      "featured_image": featured_image(browser),
      "facts": facts,
      "last_modified": dt.datetime.now(),
      "hemispheres": mars_hemispheres(browser)
    }
    
    browser.quit()
    return data



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [/Users/lindaperez/.wdm/drivers/chromedriver/mac64/100.0.4896.60/chromedriver] found in cache


True
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Mars</th>
      <th>Earth</th>
    </tr>
    <tr>
      <th>Description</th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Mars - Earth Comparison</th>
      <td>Mars</td>
      <td>Earth</td>
    </tr>
    <tr>
      <th>Diameter:</th>
      <td>6,779 km</td>
      <td>12,742 km</td>
    </tr>
    <tr>
      <th>Mass:</th>
      <td>6.39 × 10^23 kg</td>
      <td>5.97 × 10^24 kg</td>
    </tr>
    <tr>
      <th>Moons:</th>
      <td>2</td>
      <td>1</td>
    </tr>
    <tr>
      <th>Distance from Sun:</th>
      <td>227,943,824 km</td>
      <td>149,598,262 km</td>
    </tr>
    <tr>
      <th>Length of Year:</th>
      <td>687 Earth days</td>
      <td>365.24 days</td>
    </tr>
    <tr>
      <th>Temperature:</th>
      <td>-87 to -5 °C</td>
      <td>-88 to 58°C</td>
    </tr>
  </tbody>
</table>
[{'img_url': 'https://marshemispher

{'news_title': 'NASA Invites Students to Name Mars 2020 Rover',
 'news_paragraph': "Through Nov. 1, K-12 students in the U.S. are encouraged to enter an essay contest to name NASA's next Mars rover.",
 'featured_image': 'https://spaceimages-mars.com/image/featured/mars3.jpg',
 'facts': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,9

### Visit the NASA Mars News Site

In [8]:
def mars_news(browser):
    # Visit the mars nasa news site
    url = 'https://redplanetscience.com/'
    browser.visit(url)

    # Optional delay for loading the page
    waited = browser.is_element_present_by_css('div.list_text', wait_time=1)
    #print(waited)
    # Convert the browser html to a soup object and then quit the browser
    html = browser.html
    news_soup = soup(html, 'html.parser')
    try:
        slide_elem = news_soup.select_one('div.list_text')

        slide_elem.find('div', class_='content_title')

        # Use the parent element to find the first a tag and save it as `news_title`
        news_title = slide_elem.find('div', class_='content_title').get_text()

        # Use the parent element to find the paragraph text
        news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
    except AttributeError:
            return None
    return news_title, news_p

### JPL Space Images Featured Image

In [40]:
def featured_image(browser):
    # Visit URL
    url = 'https://spaceimages-mars.com'
    browser.visit(url)

    # Find and click the full image button
    full_image_elem = browser.find_by_tag('button')[1]
    full_image_elem.click()

    # Parse the resulting html with soup
    html = browser.html
    img_soup = soup(html, 'html.parser')
    
    try:
        # find the relative image url
        img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')


        # Use the base url to create an absolute url
        img_url = f'https://spaceimages-mars.com/{img_url_rel}'
    except AttributeError:
        return None
    return img_url

### Mars Facts

In [10]:
def mars_facts():
    try:
        df = pd.read_html('https://galaxyfacts-mars.com')[0]
        df.columns=['Description', 'Mars', 'Earth']
        df.set_index('Description', inplace=True)
      
    except BaseException:
        return None
    print(df.to_html())
    return df.to_html()

# D1: Scrape High-Resolution Mars’ Hemisphere Images and Titles

### Hemispheres

In [56]:
def mars_hemispheres(browser):
    # 1. Use browser to visit the URL 
    url = 'https://marshemispheres.com/'

    browser.visit(url)

    # 2. Create a list to hold the images and titles.
    hemisphere_image_urls = []

    # 3. Write code to retrieve the image urls and titles for each hemisphere.

    browser.is_element_present_by_css('div#product-section', wait_time=1)  # delay to load page
    hemisps_soup = soup(browser.html, 'html.parser')
    h_selector = hemisps_soup.select_one('div#product-section')
    h_items = h_selector.find_all('div', class_='item')


    for item in h_items:
        img_url = f"{url}{item.select_one('img.thumb').get('src')}"

        title_selector = item.select_one('a.itemLink.product-item')
        title = title_selector.find_next('h3').get_text()
        title_link = f"{url}{title_selector.get('href')}"
        img_url = find_full_image_hemispheres(url,title_link,browser)
        #print(title_link)
        hemisphere_image_urls.append({'img_url':img_url, 'title':title})

    # 4. Print the list that holds the dictionary of each image url and title.
    #print(hemisphere_image_urls)
    return hemisphere_image_urls

In [50]:
def find_full_image_hemispheres(url,title_link,browser):

    # Find and click the full image button
    browser.visit(title_link)

    # Parse the resulting html with soup
    html_img = browser.html
    img_hem_soup = soup(html_img, 'html.parser')
  
    
    try:
        # find the relative image url
        div_downloads = img_hem_soup.find('div', class_='downloads')
        full_img_src = div_downloads.find('a').get('href')


        # Use the base url to create an absolute url
        img_url = f'{url}{full_img_src}'

    except AttributeError as e:
        print(str(e))
        return None
    return img_url